In [4]:
# imports

import datetime
import math
import searchtweets
import pandas as pd

import string
import warnings
import datetime
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import seaborn as sns
from IPython.display import display
from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package stopwords to /home/keras/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
curated_list = ['AwkwardRambler','AngryBlackN8V',
             'Pandabbadon', 'LammaticHama','DeadDogLake',
             'cricketcrocker','devilishgrin000','LowArctic',
             'ChelseyMooner','akayatuk','nativeopinion',
             'BadSalishHeart','joyem_braun','RuthH_Hopkins',
             'brettachapman','NativeApprops','Pam_Palmater',
             'AzieDee','BigIndianGyasi','RoanhorseBex',
             'Leave_Matoaka','ZoeSTodd',
             'notvanishing','Dallas_Hunt','justicedanielh',
             'WordsandGuitar','tuckeve','tagaq',
             'TanyaTalaga','Hayden_King','gindaanis',
             'antalalakam','dearnonnatives','VinceSchilling',
             'DelSchilling','LowaBeebe',
             'llewellynjobs','DaveAlexRoberts','SaltyLilOjibwe',
             'DarrylLeroux','mredshirtshaw','mors_lakota',
             'Ruth4Nd','Indigenia','ItazipcoLakota',
             'ryanredcorn','KimTallBear','tanayawinder',
             'shotheekwe','msimmons444','powhatannative',
             'aliwatson117','jfkeeler','ThunderingElks',
             'FrankWaln','AgentNdn','DeLesslin']

In [3]:
print(len(curated_list))

57


In [6]:
# @justicedanielh, @KimTallBear on curated list also

I_scholars_list = ['justicedanielh', 'PepePierce', 'pollysgdaughter', 'rebeccanagle', 'KimTallBear']

In [7]:
# need to get historical tweets from curated list & scholars
# might as well do it all at once

full_list = ['AwkwardRambler','AngryBlackN8V',
             'Pandabbadon', 'LammaticHama','DeadDogLake',
             'cricketcrocker','devilishgrin000','LowArctic',
             'ChelseyMooner','akayatuk','nativeopinion',
             'BadSalishHeart','joyem_braun','RuthH_Hopkins',
             'brettachapman','NativeApprops','Pam_Palmater',
             'AzieDee','BigIndianGyasi','RoanhorseBex',
             'Leave_Matoaka','ZoeSTodd',
             'notvanishing','Dallas_Hunt','justicedanielh',
             'WordsandGuitar','tuckeve','tagaq',
             'TanyaTalaga','Hayden_King','gindaanis',
             'antalalakam','dearnonnatives','VinceSchilling',
             'DelSchilling','LowaBeebe',
             'llewellynjobs','DaveAlexRoberts','SaltyLilOjibwe',
             'DarrylLeroux','mredshirtshaw','mors_lakota',
             'Ruth4Nd','Indigenia','ItazipcoLakota',
             'ryanredcorn','KimTallBear','tanayawinder',
             'shotheekwe','msimmons444','powhatannative',
             'aliwatson117','jfkeeler','ThunderingElks',
             'FrankWaln','AgentNdn','DeLesslin',
             'PepePierce','pollysgdaughter','rebeccanagle']

In [6]:
print(len(full_list))

60


In [19]:
# version w error handling:

def get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=True):
    
    '''
    takes:
    
    * date range
    * screen name
    * number of tweets to return (500 per page)
       
    input search term as a string
    
    input search range as datetime.datetime(20xx, 1, 20)
    
    include_retweets defaults is True. to exclude retweets set to False.
    
    returns: 
    
    * tweets object with replies to input screen name 
   
    '''
    
    
    search_args = searchtweets.load_credentials('twitter_keys.yaml',
                                                yaml_key='search_tweets_api',
                                                env_overwrite=False)
    
    if include_retweets:
        
        #search_term = search_term + ' lang:en'
        
        search_term = '(to:' + search_term + ') is:reply lang:en'
    
    else:
        
        #search_term = search_term + ' lang:en -is:retweet'
        
        search_term = '(to:' + search_term + ') is:reply lang:en -is:retweet'
        
    
    
    rule = searchtweets.gen_rule_payload(search_term,
                                         results_per_call=500,
                                         from_date=start_date.strftime('%Y-%m-%d'),
                                         to_date=end_date.strftime('%Y-%m-%d'))     
        
    tweet_objects = []
    
    rs = searchtweets.ResultStream(rule_payload=rule,
                                   max_results=num_of_tweets,
                                   max_pages=math.ceil(num_of_tweets / 500.0),
                                   **search_args)
    
    start_time = time.time()
    
    # error handling & rate limiting for requests
    
    try:
        
        # generator 

        for i, tweet_object in enumerate(rs.stream(), start=1):
            
            # check every 500 requests, ie every page
        
            if i % 500 == 0:
            
                now = time.time()
                
                # four second delay
            
                if now < start_time + 4:
                
                    time.sleep(4 - (now - start_time))
                
                    start_time = time.time()
                
            tweet_objects.append(tweet_object)
    
    except Exception as e:
        
        print("Error from Twitter API: %s. Fetched %d tweet_objects" % (e, len(tweet_objects)))
    
    return tweet_objects


In [20]:
def tweet_obj_to_df(tweet_objects):
    
    tweet_dict = {'text': [],
                  'author': [],
                  'replying_to': []
                 }
    
    for tweet in tweet_objects:
        
        tweet_dict['text'].append(tweet.all_text)
        
        tweet_dict['author'].append(tweet.screen_name)
        
        tweet_dict['replying_to'].append(tweet.in_reply_to_screen_name)
    
    tweets_df = pd.DataFrame(tweet_dict)
    
    return tweets_df

In [56]:
# get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=True):

# search_term = '(to:USERNAME) is:reply'

test_tweet_object = get_tweet_object('rebeccanagle', datetime.datetime(2018, 1, 1), datetime.datetime(2020, 1, 26), 100, include_retweets=False)

test_tweets_df = tweet_obj_to_df(test_tweet_object)

print(len(test_tweets_df), '\n')

print(test_tweets_df.head(20))

Grabbing bearer token from OAUTH


100 

                                                 text           author  \
0   @rebeccanagle Sound like a Bernie Slogan... #N...  RichardFerreiro   
1                              @rebeccanagle Truth!!!      bear_daddys   
2                   @rebeccanagle That sounds “great”         jbecks74   
3   @rebeccanagle @KimTallBear @highcountrynews I ...        Jamie_Maz   
4   @rebeccanagle is there anyway you remember the...     nrtankersley   
5   @rebeccanagle @peterdaou The same publication ...  A_Savage_Indian   
6   @rebeccanagle @DianeSnavely @KimTallBear @high...   TammyKosiancic   
7   @rebeccanagle @DianeSnavely @KimTallBear @high...   TammyKosiancic   
8   @rebeccanagle Nishnabè which sort of translate...       sour_utley   
9   @rebeccanagle Kanieknehake, means People of th...  wrdswthstranger   
10  @rebeccanagle @peterdaou I knew nothing about ...       ObserverPa   
11  @rebeccanagle @highcountrynews @KimTallBear A ...           TomLeb   
12  @rebeccanagle @highcountryne

In [48]:
test_tweets_df.head(20)

,text,author,replying_to
0,@rebeccanagle Sound like a Bernie Slogan... #N...,RichardFerreiro,rebeccanagle
1,@rebeccanagle Truth!!!,bear_daddys,rebeccanagle
2,@rebeccanagle That sounds “great”,jbecks74,rebeccanagle
3,@rebeccanagle @KimTallBear @highcountrynews I ...,Jamie_Maz,rebeccanagle
4,@rebeccanagle is there anyway you remember the...,nrtankersley,rebeccanagle
5,@rebeccanagle @peterdaou The same publication ...,A_Savage_Indian,rebeccanagle
6,@rebeccanagle @DianeSnavely @KimTallBear @high...,TammyKosiancic,rebeccanagle
7,@rebeccanagle @DianeSnavely @KimTallBear @high...,TammyKosiancic,rebeccanagle
8,@rebeccanagle Nishnabè which sort of translate...,sour_utley,rebeccanagle
9,"@rebeccanagle Kanieknehake, means People of th...",wrdswthstranger,rebeccanagle


Since I'll be looking at about sixty accounts, a function will make things easier:

In [21]:
def get_raw_dataframes(search_list, start_date, end_date, num_of_tweets, include_retweets=True):
    
    '''
    takes a list of twitter screen names & date range
    returns a dataframe with replies to that screen name
    within the provided time range
    
    '''
    
    df = pd.DataFrame()
    
    for search_term in search_list:
        
        tweet_object = get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=include_retweets)
        
        tweets_raw_df = tweet_obj_to_df(tweet_object)
        
        df = df.append(tweets_raw_df, ignore_index=True)
        
        print('In reply to ', search_term, 'dataset length: ', len(tweets_raw_df), '\n')
        
        print(tweets_raw_df.head(20), '\n')
    
    return df
    

In [39]:
# get_raw_dataframes(search_list, start_date, end_date, num_of_tweets, include_retweets=True):

short_test_list = ['justicedanielh', 'PepePierce']

test_df = get_raw_dataframes(short_test_list, datetime.datetime(2007,1,1), datetime.datetime(2016,7,1), 100, include_retweets=False)

Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


In reply to  justicedanielh dataset length:  100 

                                                 text           author  \
0   @justicedanielh Yep. I just read Wild Seed.......      joyfulcarla   
1   @justicedanielh, hahaha. When I first saw that...         debreese   
2   @justicedanielh @NativeApprops I am in Thunder...         debreese   
3   @justicedanielh, no doubt that all this stuff ...         debreese   
4   @justicedanielh @NativeApprops @MissusTWalker ...         debreese   
5   @justicedanielh And then there's self-proclaim...          clclyne   
6   @justicedanielh Have you been watching CLEVERM...     annegalloway   
7   @justicedanielh also being mestizo is much eas...  ChimalpahinXIII   
8   @justicedanielh also it seems Spain with it's ...  ChimalpahinXIII   
9   @justicedanielh the eastern nations never got ...  ChimalpahinXIII   
10  @justicedanielh I guess it's also because we w...  ChimalpahinXIII   
11  @justicedanielh it's weirs huh? Even in Canada...  Chimal

### Two Date Ranges

I'll be testing 2 date ranges to try to see whether sentiment towards Natives (as measured in replies) has changed significantly over the past several years. The first period is 2007 (shortly after Twitter started) to 2016. The seond is 2016 to the present.

Within the 2016 date range, I am looking at two main "events" (although they are not exactly discrete): the 2016 pre-election cycle when Senator Elizabeth Warren was first mockingly referred to as "Pocahontas" by Donald Trump, and the election itself. 

#### Time Period I: pre-2016 Election

I'm choosing July of 2016 as a marker for when anti-Indigenous & other racist rhetoric began to escalate.

In [42]:
'''
test_df = get_raw_dataframes(short_test_list, datetime.datetime(2007,1,1), datetime.datetime(2016,7,1), 100, include_retweets=False)

'''

raw_replies_df = get_raw_dataframes(full_list, datetime.datetime(2007,1,1), datetime.datetime(2016,7,1), 1000, include_retweets=False)


Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


In reply to  AwkwardRambler dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 

In reply to  AngryBlackN8V dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 



Grabbing bearer token from OAUTH


In reply to  Pandabbadon dataset length:  1000 

                                                 text          author  \
0                         @Pandabbadon goddamnit! No!      BeckettFan   
1   @Pandabbadon there's a lot of tweets in my tim...      BeckettFan   
2   @Pandabbadon Dephine? Have we found out what h...      BeckettFan   
3                     @Pandabbadon did they kill her?      BeckettFan   
4   @Pandabbadon it's Pet Smart! I used to work th...     OhFrankABee   
5   @Pandabbadon I mean, no one there paid any min...    necromantiic   
6   @Pandabbadon Bern Notice. Bern, Baby, Bern, Di...     DarthSienna   
7   @Pandabbadon Soon you will have extensive real...     DarthSienna   
8   @Pandabbadon Nothing compares to the Great Ind...           Eat24   
9    @Pandabbadon it’s November. She’s gonna be loud!     tamarasiuda   
10  @Pandabbadon that's incredible. Yeah, I've che...    necromantiic   
11  @Pandabbadon I know!  It was just nice to see ...        ggstar71   
12

Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


...

In reply to  AgentNdn dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 



Grabbing bearer token from OAUTH


In reply to  DeLesslin dataset length:  442 

                                                 text           author  \
0   @DeLesslin #walkinginpeace means knowing your ...        skyeakbar   
1   @DeLesslin Incorrect. There are many. That is ...         SteelRaz   
2   @DeLesslin I see how it could be perceived tha...         SteelRaz   
3   @DeLesslin My words, in which I stated or impl...         SteelRaz   
4   @DeLesslin Maybe you'll learn not to prejudge ...         SteelRaz   
5   @DeLesslin You have no idea at all what I subs...         SteelRaz   
6   @DeLesslin And Warren's family is not a center...         SteelRaz   
7   @DeLesslin If she doesn't "correct" them, then...         SteelRaz   
8   @DeLesslin You are the keeper of this identity...         SteelRaz   
9   @DeLesslin I'm 'undermining' nothing. That *yo...         SteelRaz   
10  @DeLesslin You would hold someone else's misbe...         SteelRaz   
11  @DeLesslin I just think it's amusing you telli...         Stee

Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


In reply to  pollysgdaughter dataset length:  877 

                                                 text           author  \
0   @pollysgdaughter @EdMorrissey @AHMalcolm @NRDe...        opechanga   
1   No real #NDNZ freak out at, "who're your peopl...         nicdhana   
2   Warren is the one who exploited her family to ...         nicdhana   
3   @pollysgdaughter @JoeDunn8 @elizabethforma @op...      Speshlk0510   
4   @pollysgdaughter So...if she had no shame to c...       danwenzehe   
5   @pollysgdaughter oops! Mea apologia. I'se pay ...       danwenzehe   
6   @pollysgdaughter the nation should start by pr...     SandeeLovato   
7   @pollysgdaughter @realDonaldTrump  Pocahontas ...     SandeeLovato   
8   @pollysgdaughter @SenWarren Ducking and hiding...     SandeeLovato   
9   @pollysgdaughter @espinelli145 Stop picking on...  ActuallyDontGAS   
10  @pollysgdaughter @OutlawTacoMama Let's try thi...          Atagahi   
11  @pollysgdaughter I already agreed she was wron...       

In [45]:
print(len(raw_replies_df))

print(len(raw_replies_df.dropna()))

30660
30660


In [58]:
raw_replies_df.to_csv('raw_replies_2007_to_2016.csv')

#### Second Period: 2016 - Present

This is the period including both the late election cycle, election itself, & Sen. Warren's being mocked using the name of a kidnapped Indigenous child (Matoaka/Pocahontas). Anecdotal evidence suggests these events unleashed a torrent of anti-Indigenous sentiment on social media.

In [12]:
# build post-2016 corpora

raw_replies_2016_to_2020_df = get_raw_dataframes(full_list, datetime.datetime(2016,7,2), datetime.datetime(2020,1,28), 1000, include_retweets=False)


Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


In reply to  AwkwardRambler dataset length:  1000 

                                                 text          author  \
0   @AwkwardRambler Yes it is that's why I hate si...     diviniityy_   
1   @AwkwardRambler @emmatsalageek i remember seei...        teueikan   
2                               @AwkwardRambler Gross  michellechubb2   
3   @AwkwardRambler It’s so gross how white people...       UrbanInuk   
4   @AwkwardRambler i knew before i even scrolled ...   quiet_spheres   
5   @AwkwardRambler whyyyy would you post your reg...         scwawei   
6         @AwkwardRambler Bhad Bhabie ain’t slick smh      tayhowsway   
7   @AwkwardRambler Bro lmaooooooo I sho was wonde...      tayhowsway   
8   @AwkwardRambler LMAOOOOOOOOOOOOOOOOOOOOOOO did...         r3zd0ll   
9               @AwkwardRambler @LibratariusXO Puppy!     fromshadows   
10      @AwkwardRambler native twitter may never know   emmatsalageek   
11  @AwkwardRambler Is that the so called al*ino s...     GrizzlyNoaB   

Grabbing bearer token from OAUTH


In reply to  AngryBlackN8V dataset length:  866 

                                                 text           author  \
0             I aint mad it just seem performative js    AngryBlackN8V   
1   @AngryBlackN8V And apparently she been saying ...          nilanjl   
2       It cool you know now but rt our posts bout it    AngryBlackN8V   
3                             @AngryBlackN8V So hot 🔥  beckycurlymagic   
4            @AngryBlackN8V I been sayin this 4 years         papadaca   
5   @AngryBlackN8V In a couple years i predict the...         papadaca   
6   @AngryBlackN8V I could give another fuck about...     emma71954196   
7   @AngryBlackN8V They just don’t understaaaaaand...     emma71954196   
8   @AngryBlackN8V I think she’s confused about wh...        not2early   
9   @AngryBlackN8V The fact shes made like 6 posts...     Octobers_Rot   
10  @AngryBlackN8V No, just no, those aren't locs,...        cosmyra64   
11  @AngryBlackN8V Mats. She has mats. And probabl...    madam

Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


...

Error from Twitter API: . Fetched 0 tweet_objects
In reply to  PepePierce dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 



Grabbing bearer token from OAUTH
retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': '(to:pollysgdaughter) is:reply lang:en -is:retweet', 'maxResults': 500, 'toDate': '202001280000', 'fromDate': '201607020000'}


Error from Twitter API: . Fetched 0 tweet_objects
In reply to  pollysgdaughter dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 



Grabbing bearer token from OAUTH
retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': '(to:rebeccanagle) is:reply lang:en -is:retweet', 'maxResults': 500, 'toDate': '202001280000', 'fromDate': '201607020000'}


Error from Twitter API: . Fetched 0 tweet_objects
In reply to  rebeccanagle dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 



In [13]:
# save as csv

raw_replies_2016_to_2020_df.to_csv('raw_replies_2016_to_2020.csv')

In [16]:
# TODO
# 'AgentNdn','DeLesslin','PepePierce','pollysgdaughter','rebeccanagle'
# were left out due to request limits
# run this list to make sure we have a solid comparison
# add to raw_replies_2016_to_2020

In [22]:
scholars_list_2 = ['AgentNdn','DeLesslin','PepePierce','pollysgdaughter','rebeccanagle']

In [23]:
raw_scholars2_replies_2016_to_2020_df = get_raw_dataframes(scholars_list_2, datetime.datetime(2016,7,2), datetime.datetime(2020,1,28), 1000, include_retweets=False)


Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


In reply to  AgentNdn dataset length:  0 

Empty DataFrame
Columns: [text, author, replying_to]
Index: [] 



Grabbing bearer token from OAUTH


In reply to  DeLesslin dataset length:  1000 

                                                 text          author  \
0   @DeLesslin The absolute absurdity that they ha...     JayDragon07   
1   @DeLesslin I read this as the boi being on the...       ianmnoone   
2                                    JK I don’t dance       DeLesslin   
3   @DeLesslin Evangelical fashion sure has change...         Atagahi   
4                             @DeLesslin What a cult.   AngryHahtDish   
5   @DeLesslin this isn't in my wheelhouse, but se...        emmalola   
6   @DeLesslin They made a movie about us https://...   taint_behavin   
7   @DeLesslin For when you really want people to ...       ianmnoone   
8   @DeLesslin Felt compelled to look at M T gifs....         bunzy55   
9   @DeLesslin He does the judging for both of you...        Ninjapus   
10  Another favorite bio:\n\nLine 1: Arbitrary jud...       DeLesslin   
11    @DeLesslin A highly effective filter I suppose?       toxidrone   
12  

Grabbing bearer token from OAUTH


In reply to  PepePierce dataset length:  1000 

                                                 text           author  \
0   @PepePierce @Halfrican_One @DrProfBlackLady @P...      OrionKidder   
1   @PepePierce @Halfrican_One @DrProfBlackLady @P...   justicedanielh   
2   @Halfrican_One @DrProfBlackLady @PolitiekScept...       PepePierce   
3   @Halfrican_One @nickykaymichael @DrProfBlackLa...       PepePierce   
4   @PepePierce Iyko Day article "Being or Nothing...       smithmj303   
5   @PepePierce Huh?  Don't people realize Indigen...  nickykaymichael   
6   @PepePierce So I’ve been working through afrop...      SimmonsKali   
7       @PepePierce I believe it's a kind of nihilism  deepgreenphilly   
8   @PepePierce @Politics4Bernie @AlytaDeLeon @ewa...       iamJxWoods   
9              @PepePierce @ewarren You’re brilliant.   KassandraSeven   
10  @PepePierce @AlytaDeLeon @ewarren Yeah, I thin...  Politics4Bernie   
11  @PepePierce @AlytaDeLeon @ewarren Seems like "...  Politics4

Grabbing bearer token from OAUTH


In reply to  pollysgdaughter dataset length:  1000 

                                                 text          author  \
0   @pollysgdaughter @Kamalaallday Me too I'm sick...        rethe125   
1   @pollysgdaughter @WeWillRiseOhio ~CiViLiTy~\n\...       MlleKitty   
2   @pollysgdaughter @Kamalaallday They don't know...  WeWillRiseOhio   
3   @pollysgdaughter I believe it was rigged to be...     SideupSonny   
4                @pollysgdaughter Or a Russian asset.  longtimeonleft   
5   @pollysgdaughter... I don't know how you do it...    sowersgarden   
6                               @pollysgdaughter same      serenebabe   
7   @pollysgdaughter I watched a few clips, but it...    zacrussell93   
8   @pollysgdaughter The ICWA must be saved. I am ...        waligore   
9                         @pollysgdaughter I hear ya.          ARWXCH   
10  @pollysgdaughter Yep. Or a Trump supporter. \n...  GGforthepeople   
11  @pollysgdaughter Damned if you do. Damned if y...   JaquxyzXaiXai  

In [24]:
raw_scholars2_replies_2016_to_2020_df.to_csv('raw_scholars2_replies_2016_to_2020_df.csv')

In [26]:
# reload original csv & append new names

raw_replies_df_reload = pd.read_csv('raw_replies_2016_to_2020.csv')

raw_replies_df_reload.append(raw_scholars2_replies_2016_to_2020_df)

,Unnamed: 0,author,replying_to,text
0,0.0,diviniityy_,AwkwardRambler,@AwkwardRambler Yes it is that's why I hate si...
1,1.0,teueikan,AwkwardRambler,@AwkwardRambler @emmatsalageek i remember seei...
2,2.0,michellechubb2,AwkwardRambler,@AwkwardRambler Gross
3,3.0,UrbanInuk,AwkwardRambler,@AwkwardRambler It’s so gross how white people...
4,4.0,quiet_spheres,AwkwardRambler,@AwkwardRambler i knew before i even scrolled ...
...,...,...,...,...
3995,NaN,TheronBowles,rebeccanagle,@rebeccanagle Heard it went east side of Tahle...
3996,NaN,milanos_best,rebeccanagle,@rebeccanagle @troylittledeer I hope this is a...
3997,NaN,biophiliaceae,rebeccanagle,"@rebeccanagle Thundering here too, just rain s..."
3998,NaN,AngryKavanaugh,rebeccanagle,@rebeccanagle My mom always said these were th...


In [27]:
raw_replies_df_reload.to_csv('raw_replies_2016_to_2020_full.csv')